In [1]:
import pickle,os,sys,gc
import pandas as pd
import numpy as np
import ssdeep
import glob
from tqdm import tqdm
import functools,operator

一個禮拜中某isp各protocols的session總數

In [46]:
isp = '台灣固網'
date_li = ["20200106","20200107","20200108","20200109","20200110","20200111","20200112"]  #改!!
file_name = "_".join(sorted(date_li)) #date_li 改!!
min_date = str(min(date_li))
max_date = str(max(date_li))

In [47]:
noise_path = '/home/antslab/NAS1_RAID6/pcap_inter/'+str(min_date[:4])+'_'+str(min_date[4:6])+'_'+str(min_date[6:])+'/'+isp+'/case_pickles/noise_cluster.pkl'
denoise_path = '/home/antslab/NAS1_RAID6/pcap_inter/'+str(min_date[:4])+'_'+str(min_date[4:6])+'_'+str(min_date[6:])+'/'+isp+'/case_pickles/denoise_cluster.pkl'
pickle_dir = '/home/antslab/NAS1_RAID6/pcap_inter/'+str(min_date[:4])+'_'+str(min_date[4:6])+'_'+str(min_date[6:])+'/'+isp+'/case_pickles/'
all_df = pickle.load(open(pickle_dir+'clusterName_overview_denoise_df_'+str(min_date)+'_'+str(date_li[-1])+'.pkl','rb'))
noise_clusters = pickle.load(open(noise_path,'rb'))
denoise_clusters = pickle.load(open(denoise_path,'rb'))
assert len(noise_clusters) + len(denoise_clusters) == len(all_df)
all_df = all_df[~all_df.idx.isin(noise_clusters)]
# all_df = all_df[all_df.idx.isin(denoise_clusters)]
# all_df = all_df.reset_index(drop=True)
all_df

,idx,timestamp,country,src_ip,ssdeep
0,20200106_15733_http,"[1578239430.546177, 1578239430.627715, 1578239...","[United States, United States, United States, ...","[66.249.69.175, 66.249.69.145, 66.249.71.5, 66...",[48:+eWa5lO918hw18VcM18xdnXtctdLJCAY6S:+eXg91u...
2,20200106_177_http,"[1578239428.145726, 1578239432.99058, 15782394...","[Netherlands, United States, Iran, United Stat...","[169.197.108.38, 216.244.66.198, 217.60.228.17...",[12:rHXq6xKv89IshSR56Y1awh5UPlVFYNjZpvhm:Da5kW...
3,20200106_51539642503_http,"[1578248664.635621, 1578248842.672464, 1578249...","[Hong Kong, Egypt, Hong Kong, China, China, Ch...","[103.90.203.174, 193.227.5.230, 103.90.203.174...",[12:7KS3TXcHnMQ7s95HfUzUfszUfsYjmHXq/Ish+j51h1...
6,20200106_136_http,"[1578239430.724066, 1578239431.97968, 15782394...","[China, China, China, Taiwan, China, China, Ch...","[106.13.108.181, 59.38.126.238, 59.38.126.238,...",[12:7QoEmHXq/I75cuXNFljhgDxKv8rDH46ZY1Zu8m:7UA...
7,20200106_2386_http,"[1578241539.855242, 1578241926.134562, 1578241...","[United States, France, France, France, France...","[108.52.243.101, 147.135.130.142, 147.135.130....",[12:7iotDpiNOmHXqzNkZr1NkWQwHj1NkZfsvq7ALTbxKv...
...,...,...,...,...,...
630,20200110_42949697374_telnet,"[1578641181.71725, 1578664950.870665]","[United States, United States]","[204.93.154.208, 204.93.154.194]","[3:ro53fg5T:ro2N, 3:ro53fg5T:ro2N, 3:ro53fg5T:..."
631,20200108_94489305705_sip,"[1578455241.326382, 1578455241.327904, 1578455...","[France, France, France, France, France, Franc...","[5.135.73.105, 5.135.73.105, 5.135.73.105, 5.1...",[24:p2kQ17mDETU7ft9ckQFAOckQFA+UDR0dSRtPKiRvb3...
632,20200106_10024_sip,"[1578293821.543121, 1578293821.543636, 1578293...","[Estonia, Estonia, Estonia, Estonia, Estonia, ...","[37.49.230.91, 37.49.230.91, 37.49.230.91, 37....",[48:Yoaj0rXuXugIAO08oKY18Vje821BCkOPk/Ix:YnoXu...
633,20200106_1876_sip,"[1578239432.53756, 1578239442.108728, 15782394...","[United States, United States, United States, ...","[104.206.128.22, 104.206.128.22, 104.140.188.2...",[12:qYlim7mDE4Az5g9qX6oa+sCssVWGY07GiDv+5gTXNk...


In [48]:
def cut_protocol(cluster_id):
    '''
    GOAL: 將cluster index轉換為protocol名稱
    '''
    return cluster_id.split("_")[-1]
def count_sessions_num(time_li):
    '''
    GOAL: 依據timestamp list計算每個cluster所具有的session數量
    '''
    return len(time_li)
all_df['protocol'] = all_df.idx.map(cut_protocol)
all_df['session_num'] = all_df.timestamp.map(count_sessions_num)
all_df

,idx,timestamp,country,src_ip,ssdeep,protocol,session_num
0,20200106_15733_http,"[1578239430.546177, 1578239430.627715, 1578239...","[United States, United States, United States, ...","[66.249.69.175, 66.249.69.145, 66.249.71.5, 66...",[48:+eWa5lO918hw18VcM18xdnXtctdLJCAY6S:+eXg91u...,http,331276
2,20200106_177_http,"[1578239428.145726, 1578239432.99058, 15782394...","[Netherlands, United States, Iran, United Stat...","[169.197.108.38, 216.244.66.198, 217.60.228.17...",[12:rHXq6xKv89IshSR56Y1awh5UPlVFYNjZpvhm:Da5kW...,http,206866
3,20200106_51539642503_http,"[1578248664.635621, 1578248842.672464, 1578249...","[Hong Kong, Egypt, Hong Kong, China, China, Ch...","[103.90.203.174, 193.227.5.230, 103.90.203.174...",[12:7KS3TXcHnMQ7s95HfUzUfszUfsYjmHXq/Ish+j51h1...,http,193379
6,20200106_136_http,"[1578239430.724066, 1578239431.97968, 15782394...","[China, China, China, Taiwan, China, China, Ch...","[106.13.108.181, 59.38.126.238, 59.38.126.238,...",[12:7QoEmHXq/I75cuXNFljhgDxKv8rDH46ZY1Zu8m:7UA...,http,46402
7,20200106_2386_http,"[1578241539.855242, 1578241926.134562, 1578241...","[United States, France, France, France, France...","[108.52.243.101, 147.135.130.142, 147.135.130....",[12:7iotDpiNOmHXqzNkZr1NkWQwHj1NkZfsvq7ALTbxKv...,http,33540
...,...,...,...,...,...,...,...
630,20200110_42949697374_telnet,"[1578641181.71725, 1578664950.870665]","[United States, United States]","[204.93.154.208, 204.93.154.194]","[3:ro53fg5T:ro2N, 3:ro53fg5T:ro2N, 3:ro53fg5T:...",telnet,2
631,20200108_94489305705_sip,"[1578455241.326382, 1578455241.327904, 1578455...","[France, France, France, France, France, Franc...","[5.135.73.105, 5.135.73.105, 5.135.73.105, 5.1...",[24:p2kQ17mDETU7ft9ckQFAOckQFA+UDR0dSRtPKiRvb3...,sip,3901
632,20200106_10024_sip,"[1578293821.543121, 1578293821.543636, 1578293...","[Estonia, Estonia, Estonia, Estonia, Estonia, ...","[37.49.230.91, 37.49.230.91, 37.49.230.91, 37....",[48:Yoaj0rXuXugIAO08oKY18Vje821BCkOPk/Ix:YnoXu...,sip,2277
633,20200106_1876_sip,"[1578239432.53756, 1578239442.108728, 15782394...","[United States, United States, United States, ...","[104.206.128.22, 104.206.128.22, 104.140.188.2...",[12:qYlim7mDE4Az5g9qX6oa+sCssVWGY07GiDv+5gTXNk...,sip,1917


In [49]:
print(isp,"經濾除noiseClusters後在",min_date,"至",max_date,"的期間中各protocols的session數量:")
#all_df.groupby("protocol")['timestamp'].sum().apply(len) #alternative
temp = pd.DataFrame(all_df.groupby('protocol')['session_num'].sum()).sort_values('session_num',ascending=False)
temp#.reset_index()

台灣固網 經濾除noiseClusters後在 20200106 至 20200112 的期間中各protocols的session數量:


,session_num
protocol,
ssh,9432064
smb,1864386
http,993223
smtp,131451
ftp,44616
mysql,26498
pop,9295
rpc,9150
sip,8153


跨ISP的ssdeep相似度比較與關聯

In [53]:
isp_li = ['中嘉寬頻','亞太電信','台固媒體','台灣固網','台灣大哥大','遠傳電信','中華電信','凱擘','台灣之星',
        '台灣基礎開發','台灣碩網']
date_li = ["20200106","20200107","20200108","20200109","20200110","20200111","20200112"]
protocols_need = ['http','mysql','ftp','smb','smtp','imap','pop','rpc','ssh','telnet','sip']
min_date = str(min(date_li))
# da_prefix = +'_'+max(date_li)
root_dir = '/home/antslab/NAS1_RAID6/pcap_inter/'
prefix = root_dir + min_date[:4]+'_'+min_date[4:6]+'_'+min_date[6:]+'/'
pickle_postfix = '_clusterID_time_country_ip_ssdeep.pkl'

all_df_li = glob.glob(prefix+'*/*/*'+pickle_postfix)

In [50]:
%%time
for i,path in enumerate(all_df_li):
    isp = path.split('/')[6]
    if i == 0 :
        all_df = pickle.load(open(path,'rb'))
        all_df['isp'] = isp
    else:
        temp = pickle.load(open(path,'rb'))
        temp['isp'] = isp
        all_df = all_df.append(temp)
all_df = all_df.reset_index(drop=True)
all_df['session_num'] = all_df.timestamp.apply(len)
all_df['protocol'] = all_df.idx.apply(lambda x:x.split('_')[-1])
all_df

CPU times: user 13.3 s, sys: 3.51 s, total: 16.9 s
Wall time: 16.9 s


,idx,timestamp,country,src_ip,ssdeep,lalo,isp,protocol,session_num
0,20200106_13656_http,"[1578239437.622562, 1578239439.00718, 15782394...","[United States, United States, United States, ...","[66.249.65.188, 66.249.79.137, 66.249.71.43, 6...",[48:+v/HamO918hw18VcM18xdnXtctdLJCAY6S:+v691uw...,"[(37.751, -97.822), (37.751, -97.822), (37.751...",亞太電信,http,183041
1,20200106_295_http,"[1578239440.513196, 1578239443.592105, 1578239...","[Iran, Russia, United States, United States, R...","[79.132.201.185, 212.41.14.170, 216.244.66.245...",[12:rHXq6xmMVkExShI/XES+AoGY1uqh5U9Z1J3vhm:Da7...,"[(35.698, 51.4115), (56.003, 92.7993), (47.722...",亞太電信,http,81808
2,20200106_2984_http,"[1578239434.537772, 1578239437.816006, 1578239...","[Russia, None, United States, United States, U...","[5.188.84.45, 109.70.100.27, 66.249.68.16, 66....",[24:tAxtAKAFMVvoCYEQTcZMzP2tS18YH+w18YHxKt1Hqa...,"[(55.7386, 37.6068), (None, None), (37.751, -9...",亞太電信,http,64609
3,20200106_51539658953_http,"[1578243129.379452, 1578243147.033231, 1578243...","[China, China, China, China, China, China, Hon...","[103.140.126.152, 103.140.126.152, 103.140.126...",[12:72wXcHnMQ7s95HfSBBBBBBBSjmHXq/Ish+j51h1JcZ...,"[(34.7725, 113.7266), (34.7725, 113.7266), (34...",亞太電信,http,48240
4,20200106_152_http,"[1578239434.511962, 1578239437.425728, 1578239...","[China, China, China, China, China, China, Chi...","[122.14.208.63, 139.199.74.166, 148.70.163.11,...",[12:/ADXEmHXq/I75cuXNFljI4DxmMVXqH46ZY1Zu8m:zA...,"[(23.1167, 113.25), (34.7725, 113.7266), (34.7...",亞太電信,http,31897
...,...,...,...,...,...,...,...,...,...
4809,20200106_42949704991_sip,"[1578293880.656561, 1578293880.66286, 15782938...","[Estonia, Estonia, Estonia, Estonia, Estonia, ...","[37.49.230.91, 37.49.230.91, 37.49.230.91, 37....",[24:pedNtuhnz0OJdNtUJdNt7AmDED83kTibWPiF83/BZs...,"[(59.0, 26.0), (59.0, 26.0), (59.0, 26.0), (59...",中嘉寬頻,sip,11906
4810,20200106_17559_sip,"[1578239563.410876, 1578239607.984642, 1578239...","[United States, United States, United States, ...","[104.206.128.58, 104.140.188.34, 170.130.187.3...",[12:qYlim7mDLRfAaAz85g9qX6oa+sCssVWGY07GiDv+5g...,"[(37.751, -97.822), (37.751, -97.822), (34.054...",中嘉寬頻,sip,447
4811,20200106_17467_sip,"[1578258055.286932, 1578258055.328613, 1578258...","[Estonia, Estonia, Estonia, Estonia, Estonia, ...","[77.247.110.179, 77.247.110.179, 77.247.110.17...",[24:sBd8Nc7S7mDEUd1gVgjmEtKimBd8Nc7gCbd8Nc7Ke2...,"[(59.0, 26.0), (59.0, 26.0), (59.0, 26.0), (59...",中嘉寬頻,sip,64
4812,20200109_2562_sip,"[1578577345.509497, 1578577345.723031, 1578577...","[China, China, China, China, China, China, Chi...","[115.233.218.203, 115.233.218.203, 115.233.218...",[24:qgcF8EL7mDE70a2ftSF4KinF8Eyd+UZF8EFymZ0JEC...,"[(30.294, 120.1619), (30.294, 120.1619), (30.2...",中嘉寬頻,sip,60


分protocols
把key拆成一個df
把values拆成拆成一個df
key的col去map一個function，function裡面是拿values df去map另一個function，回傳df
把df list merge by idx，挑出平均分數最高的給兩個群合併，若是最大分數=0則保留不做動作

取mean取max再取mean
1. 一個key ssdeep比較某value cluster的所有ssdeep，然後取平均 [sim_score]
    * 每個key ssdeep都可以獲得跟全部cluster相比的分數
2. 每個key ssdeep都取出分數最大的cluster id和分數 [calc_sim_score]
3. 把對應到cluster id的分數取出來計算平均
    * 同時考量數量(總和一分按比例分配)和分數再平均得到最終分數
4. 最終分數最高者進行歸類合併

In [75]:
pd.DataFrame(need_df.loc[1:,'ssdeep'])

,ssdeep
1,[24:pSAc1hnz09Ac6AcTUnQmDED8XUi1xbe/jv8XUi1EBZ...
2,[48:0U5La8En3uU5NPU5O39elU08oKY18Vje821BCkOPk/...
3,[24:sAnWPEgsp5k7mDEUd1dgF2EKimAnWPEgspYCSnWPEg...
4,[24:pSMfChnz09MfXMffnQmDED8Xzg1ge0v8Xz/BZsKiXY...
5,[24:qgOf3vI7mDL5gizTKiqf3vX4v/DRt0UUgbYMfy1wUX...
6,[24:sPLTON5F7mDEUd1TjgMRPwKimPLTON5NC9LTON51ef...
7,[48:HPpaeuP1MCJgFeU08oKY18Vje821BCkOPk/Ix:HP0e...
8,[24:qgcF8EL7mDE70a2ftSF4KinF8Eyd+UZF8EFymZ0JEC...
9,[24:pSMgZY+7mDETjO1AhLLJYFHDGodd8LMgZYOdd8LMgZ...
10,[24:pSmhnz0UKD+mDED8M13Hc8DBZsKiXYEXl8XQpkCEI:...


In [ ]:

def calc_sim_score(key_hash,hashli_df):
    def sim_score(hash_li,key_hash):
        def score(value_hash,key_hash):
            return ssdeep.compare(key_hash,value_hash)
        hash_df = pd.DataFrame(hash_li)
        hash_df[1] = hash_df[0].apply(score,args=(key_hash,))
        return hash_df[1].mean()
    hashli_df['mean_score']=hashli_df['ssdeep'].apply(sim_score,args=(key_hash,))
    max_score = hashli_df['mean_score'].max()
    if max_score>0:
        return [hashli_df['mean_score'].idxmax(),max_score]
    else:
        return ['NAN',max_score]

combine_dict = {}
for proto in tqdm(protocols_need): #分protocol
    need_df = all_df[all_df.protocol == proto] #分protocol
    need_df = need_df.reset_index(drop=True)
    for i in range(len(need_df)-1):
        key_hash_df = pd.DataFrame(need_df.loc[i,'ssdeep']) #a ssdeep string
        value_hash_df = pd.DataFrame(need_df.loc[i+1:,'ssdeep']) #list of ssdeep stings
        key_hash_df[1] = key_hash_df[0].apply(calc_sim_score,args=(value_hash_df,))
        tmp = key_hash_df[1].apply(pd.Series)
        kk1 = tmp.groupby([0]).count()
        kk2 = tmp.groupby([0]).mean()
        kk = pd.merge(kk1,kk2,left_index=True, right_index=True)
        kk['score1'] = kk['1_x']/kk['1_x'].sum()*100
        kk['score'] = (kk['1_y']+kk['score1'])/2
        combine_idx = kk['score'].idxmax()
        if (kk['score'].max()>0)and(combine_idx!='NAN'): #此key有相似的cluster可合併(thr可調整)
            combine_idx = int(combine_idx)
            now_val = set(functools.reduce(operator.iconcat, list(combine_dict.values()), []))
            if (i in now_val) and (combine_idx in now_val):
                flag = 'ic' #兩個都在。一種是兩個在一起，或是一人分處一邊
                for k,v in combine_dict.items():
                    if (i in v)and(combine_idx not in v):
                        if flag=='c': #代表在c已經合併過，要把這邊的都挪過去
                            tmp = combine_dict[key][:]
                            tmp.extend(v)
                            combine_dict[key] = tmp
                            combine_dict[k] = []
                            break
                        else: #代表首先遇到i
                            flag='i'
                            key = k
                    if (combine_idx in v)and(i not in v):
                        if flag == 'i': #代表i已經先合併過了
                            tmp = combine_dict[key][:]
                            tmp.extend(v)
                            combine_dict[key] = tmp
                            combine_dict[k] = []
                            break
                        else:
                            flag='c'
                            key = k
            elif i in now_val:
#                 flag = 'i' #僅i在
                for k,v in combine_dict.items():
                    if i in v:
                        tmp = v[:]
                        tmp.append(combine_idx)
                        combine_dict[k] = tmp
                        break
                        
            elif combine_idx in now_val:
#                 flag = 'c' #僅combine_idx在
                for k,v in combine_dict.items():
                    if combine_idx in v:
                        tmp = v[:]
                        tmp.append(i)
                        combine_dict[k]=tmp
                        break
            else: #兩個都不在
                combine_dict[i] = [i,combine_idx]
                
                
                
combine_dict    

 36%|███▋      | 4/11 [1:58:47<4:25:58, 2279.77s/it] 

In [167]:
combine_dict

True

In [151]:
# key_hash_df[[2,3]] = pd.DataFrame(key_hash_df[1].values.tolist(), index=key_hash_df)
# key_hash_df[[2,3]] = key_hash_df[1].str.split(expand=True)
# tmp = key_hash_df[1].apply(pd.Series)
# pd.merge(key_hash_df,tmp,left_index=True, right_index=True)
# key_hash_df
# kk1 = tmp.groupby([0]).count()
# kk2 = tmp.groupby([0]).mean()
# kk = pd.merge(kk1,kk2,left_index=True, right_index=True)
# kk['score1'] = kk['1_x']/kk['1_x'].sum()*100
# kk['score'] = (kk['1_y']+kk['score1'])/2
# kk['score'].idxmax()
# print(kk1)
# kk2
# kk
li = ['20200106_295_http','20200106_2984_http']
kk = need_df.groupby(need_df.idx.isin(li)).agg(list)
kk = kk[kk.index==True] 
kk = kk.reset_index(drop=True)
kk

,idx,timestamp,country,src_ip,ssdeep,lalo,isp,protocol,session_num
0,"[20200106_295_http, 20200106_2984_http]","[[1578239440.513196, 1578239443.592105, 157823...","[[Iran, Russia, United States, United States, ...","[[79.132.201.185, 212.41.14.170, 216.244.66.24...",[[12:rHXq6xmMVkExShI/XES+AoGY1uqh5U9Z1J3vhm:Da...,"[[(35.698, 51.4115), (56.003, 92.7993), (47.72...","[亞太電信, 亞太電信]","[http, http]","[81808, 64609]"


In [149]:
kkk = need_df[need_df.idx == '20200106_13656_http']


,idx,timestamp,country,src_ip,ssdeep,lalo,isp,protocol,session_num
0,20200106_13656_http,"[1578239437.622562, 1578239439.00718, 15782394...","[United States, United States, United States, ...","[66.249.65.188, 66.249.79.137, 66.249.71.43, 6...",[48:+v/HamO918hw18VcM18xdnXtctdLJCAY6S:+v691uw...,"[(37.751, -97.822), (37.751, -97.822), (37.751...",亞太電信,http,183041


In [159]:
yy = {1:[2,3,4],2:[5,6,7]}
sum(list(yy.values()),[])

[2, 3, 4, 5, 6, 7]

In [124]:
ssdeep.compare('3:OXXUD8EU1V1V1V1RYP/LfIQfdplr+Mfjfun:O0DVPxdL+Mfjmn','3:OXXXL1Vf1V1V1V1RYP/LfIQfdplr+M5DcEYVV91v/vV1V1VV:OQPxdL+M5Avf9')

35

In [126]:
ssdeep.compare('48:+v/HamO918hw18VcM18xdnXtctdLJCAY6S:+v691uw1Gp1sdnXiftD+','48:+Ka7UzEO918hw18VcM18xdnXtctdLJCAY6S:+do91uw1Gp1sdnXiftD+')

91

In [97]:
need_df.loc[0,'ssdeep']

['48:+v/HamO918hw18VcM18xdnXtctdLJCAY6S:+v691uw1Gp1sdnXiftD+',
 '24:tA8SNcXoa7MVXPt1Hy+MVX/DEA8S2CTjeN1dqgrFxtCBtDhLcGofALCafC6Bo:+TVa7UdUb/K7dnXtctdLJCAY6S',
 '48:+Ka7UzEO918hw18VcM18xdnXtctdLJCAY6S:+do91uw1Gp1sdnXiftD+',
 '48:+tqwa7UfO918hw18VcM18xdnXtctdLJCAY6S:+teP91uw1Gp1sdnXiftD+',
 '48:+za7UhO918hw18VcM18xdnXtctdLJCAY6S:+gJ91uw1Gp1sdnXiftD+',
 '24:tAtucXoaPU+rt1HyT+sAt8KFCTjeN1dqgrFxtCBtDhLcGofALCafC6Bo:+tuVa1tZb7dnXtctdLJCAY6S',
 '12:7XiS9HXq6xmo/+RXDHCFNkDTg1IEcuzGpfAvTYXNkZfsv6ULmXJm:7S2aPU+BCTGTgeEcGofALCafC6Bo',
 '12:7XiS9HXq6xmo/+R+pHCFNkDTg1IEcuzGpfAvTYXNkZfsv6ULmXJm:7S2aPU+WCTGTgeEcGofALCafC6Bo',
 '12:7XiS9HXq6xmo/+R7HCFNkDTg1IEcuzGpfAvTYXNkZfsv6ULmXJm:7S2aPU+xCTGTgeEcGofALCafC6Bo',
 '48:+Qa/WO918hw18VcM18xdnXtctdLJCAY6S:+D91uw1Gp1sdnXiftD+',
 '48:+7aa7UhO918hw18VcM18xdnXtctdLJCAY6S:+BJ91uw1Gp1sdnXiftD+',
 '48:+Gia7UQO918hw18VcM18xdnXtctdLJCAY6S:+sU91uw1Gp1sdnXiftD+',
 '12:7XiS9HXq6xmo/+RFHCFNkDTg1IEcuzGpfAvTYXNkZfsv6ULmXJm:7S2aPU+bCTGTgeEcGofALCafC6Bo',
 '12:7XiS9HXq6xm

In [86]:
need_df[need_df.session_num ==need_df.session_num.max()]

,idx,timestamp,country,src_ip,ssdeep,lalo,isp,protocol,session_num
19,20200108_3875_sip,"[1578432989.625082, 1578432989.625162, 1578432...","[France, France, France, France, France, Franc...","[5.135.73.105, 5.135.73.105, 5.135.73.105, 5.1...",[24:pv7mDETJ1tHPW8gJR0YfYJPQKiR+b3Za7gcXYEXl8X...,"[(48.8582, 2.3387), (48.8582, 2.3387), (48.858...",台灣碩網,sip,67261


In [ ]:
20200106_20200112_clusterID_time_country_ip_ssdeep.pkl
20200106_20200112_clusterID_time_country_ip_ssdeep.pkl

In [ ]:
2020_01_06/中嘉寬頻/case_pickles/20200106_20200112

alignment

In [3]:
from skbio.alignment import StripedSmithWaterman

In [4]:
target_sequences = [
...     "GCTAACTAGGCTCCCTTCTACCCCTCTCAGAGA",
...     "GCCCAGTAGCTTCCCAATATGAGAGCATCAATTGTAGATCGGGCC",
...     "TCTATAAGATTCCGCATGCGTTACTTATAAGATGTCTCAACGG",
...     "TAGAGATTAATTGCCACTGCCAAAATTCTG"
... ]

In [5]:
alignments = []

In [7]:
alignment, score, start_end_positions = local_pairwise_align_ssw(
...     DNA("ACTAAGGCTCTCTACCCCTCTCAGAGA"),
...     DNA("ACTAAGGCTCCTAACCCCCTTTTCTCAGA")
... )

NameError: name 'DNA' is not defined